# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

__В датасете представлена информация о заемщиках.__
Общее количество строк датасета составляет _21525_ строк.

__Следующие столбцы имеют меньше элементов чем строк в датасете:__
* `days_employed` — общий трудовой стаж в днях ***19351***;
* `total_income` — ежемесячный доход ***19351***.


По какой-то причине отсутсвует одинаковое количество данных, совпадение или умысел?

__Так-же вызывают вопросы данные:__
* `days_employed` — общий трудовой стаж в днях ***почему он отрицательный***;
* `dob_years` — возраст клиента в годах ***почему он float64 и можно ли его сделать int64***.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# Посмотрим общее количесво пустых строк в столбцах
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# Возможно есть способо проще сделать маркировку строк имеющих пустые ячейки, но я его не искал.
# Промаркируем строки имеющие пустые ячейки 
# для этого создадим столбец null и поставим True если в строке есть пустая ячейка
def column_is_null(row):
    result = False
    for column in row.index:
        if pd.isna(row[column]):
            result = True
    return result
df['null'] = df.apply(column_is_null, axis=1)

# Посмотрим на строки с пустыми ячейками
df.loc[df.null == True]
# т.к. пропуски присутсвуют в одних и тех же строках
# то вероятнее всего это или не правильная выгрузка, или тот кто заполнял умышленно пропускал
# или это была форма с возможностью пропускать эти поля.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,null
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,True
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,True
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,True
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,True
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем,True
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба,True
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости,True
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости,True


In [5]:
# Будем думать, что на трудовой стаж связан с возрастом.
# Посчитаем средний трудовой стаж для возрастов.
# Предварительно сделаем положительными отрицательные значения
df['days_employed'] = abs(df['days_employed'])
mean_days_employed = df.loc[df.null == False].groupby('dob_years')['days_employed'].mean()
#присвоим средний трудовой стаж пропущеным значениям в соответсвии с возрастом
for row in range(df.shape[0]):
    if pd.isna(df.iloc[row, 1]):
        df.iloc[row, 1] = mean_days_employed[df.iloc[row,2]]
        
#Будем думать, что доход связан с должностью
mean_total_income = df.loc[df.null == False].groupby('income_type')['total_income'].mean()
#присвоим средний доход пропущеным значениям в соответсвии с должностью
for row in range(df.shape[0]):
    if pd.isna(df.iloc[row, 10]) == True:
        df.iloc[row, 10] = mean_total_income[df.iloc[row,8]]
# Переведем дни в годы
df['year_employed'] = df['days_employed'] / 365
df['year_employed'] = df['year_employed'].astype('int')
# Проверим всели нулевые значения мы заменили
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
null                0
year_employed       0
dtype: int64

In [6]:
# Проверим есть ли люди у которых общий трудовой стаж превосходит их возраст минус 14 лет
def highlander(row):
    result = False
    if row['year_employed'] > row['dob_years'] - 14 :
            result = True
    return result
df['highlander'] = df.apply(highlander, axis=1)
df.loc[df['highlander'] == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,null,year_employed,highlander
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,False,932,True
12,0,287448.538552,65,среднее,1,гражданский брак,1,M,пенсионер,0,137127.465690,сыграть свадьбу,True,787,True
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,False,1096,True
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,False,927,True
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,False,996,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу,False,928,True
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость,False,1058,True
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью,False,992,True
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем,False,1024,True


In [7]:
# Ого 4553 человек смогли нафармить опыта больше чем прожили снимаю шляпу молодцы, но что-то тут не то.
# Недолго яндексив понял, что скорее всего это часы. Ну вот все переделывать.
def hours_to_day(row):
    result = row['days_employed']
    if row['year_employed'] > row['dob_years'] - 14 :
            result = int(row['days_employed'] / 60)
    return result
df['days_employed'] = df.apply(hours_to_day, axis=1)

In [8]:
#перепроверим
df['year_employed'] = df['days_employed'] / 365
df['year_employed'] = df['year_employed'].astype('int')
df['highlander'] = df.apply(highlander, axis=1)
df.loc[df['highlander'] == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,null,year_employed,highlander
99,0,5775.0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль,False,15,True
149,0,44.0,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем,False,0,True
270,3,31.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью,False,0,True
578,0,6630.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,False,18,True
1040,0,19.0,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль,False,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1158.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,161380.260488,жилье,True,3,True
20462,0,5645.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья,False,15,True
20577,0,5529.0,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость,False,15,True
21179,2,1.0,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости,False,0,True


In [9]:
# Отлично есть люди которым 0 лет скорее всего просто забыли вписать их возраст
# Буду думать, что на dob_years влияет income_type
mean_dob_years = df.groupby('income_type')['dob_years'].mean()
for row in range(df.shape[0]):
    if df.iloc[row, 2] == 0:
        df.iloc[row, 2] = int(mean_dob_years[df.iloc[row,8]])

**Вывод**

Все пропущенные данные были заполнены средними значениями в своих группах, что повысить точность прогноза в дальнейшем.

### Замена типа данных

In [10]:
# Для замены типа данных используем astype т.к. его можно применять сразу к столбцу
try:
    df['days_employed'] = df['days_employed'].astype(int)
    df['total_income'] = df['total_income'].astype(int)
except:
    print('возможно есть пустые строки')

**Вывод**

Количесво дней трудового стажа `days_employed` и ежемесячный доход `total_income` лучше хранить в целых так они будут занимать меньше места, а их дробная часть погоды не сыграет.

### Обработка дубликатов

In [11]:
# Использую drop_duplicates для удаления полных дублей строк в датасете.
# Дубликаты могли появится в связи с ошибками ввода, 
# пропущенными и потом заполнеными данными, человеком приходившим несколько раз в разные филиалы
# В столбцах дубликаты могли появится 
df.drop_duplicates(inplace=True)
df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 15 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
null                21471 non-null bool
year_employed       21471 non-null int64
highlander          21471 non-null bool
dtypes: bool(2), int64(8), object(5)
memory usage: 2.3+ MB


**Вывод**

Удалим строки с полными дублями. Количесво строк после удаления уменьшилось.

### Лемматизация

In [12]:
# Я не зна как описать процесс Лемматизации.
# Но я взял фразы лемматизировал удалил переносы строк и превратил в строку
from pymystem3 import Mystem
m = Mystem()
def lem_ctw(row):
    return ''.join(m.lemmatize(row)).strip( '\n' )
df['lemma_purpose'] = df['purpose'].apply(lem_ctw)
df['education_purpose'] = df['education'].apply(lem_ctw)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,null,year_employed,highlander,lemma_purpose,education_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,False,23,False,покупка жилье,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,False,11,False,приобретение автомобиль,средний
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,False,15,False,покупка жилье,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,False,11,False,дополнительный образование,средний
4,0,5671,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,False,15,False,сыграть свадьба,средний
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,False,12,False,операция с жилье,средний
21521,0,5732,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,False,15,False,сделка с автомобиль,средний
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,False,5,False,недвижимость,средний
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,False,8,False,на покупка свой автомобиль,средний


**Вывод**

Получим леммы `purpose` — цель получения кредита, чтобы было проще искать на каие цели получали кредит.
Также преобразуем `education` — уровень образования клиента,чтобы было проще искать группы по образованию.

### Категоризация данных

In [13]:
# Получим категории
# Возьмём значения от минимального до максимального с шагом в 2 раза больше минимального.
counter = range(df['total_income'].min(),df['total_income'].max(),df['total_income'].min()*2)
def get_nearest_value(value):
    return min(counter, key=lambda x: abs(x - value))
df['pepole_class'] = df['total_income'].apply(get_nearest_value)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,null,year_employed,highlander,lemma_purpose,education_purpose,pepole_class
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,False,23,False,покупка жилье,высокий,268671
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,False,11,False,приобретение автомобиль,средний,103335
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,False,15,False,покупка жилье,средний,144669
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,False,11,False,дополнительный образование,средний,268671
4,0,5671,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,False,15,False,сыграть свадьба,средний,144669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,False,12,False,операция с жилье,средний,227337
21521,0,5732,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,False,15,False,сделка с автомобиль,средний,144669
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,False,5,False,недвижимость,средний,103335
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,False,8,False,на покупка свой автомобиль,средний,227337


In [14]:
df.groupby('pepole_class')['pepole_class'].count()

pepole_class
20667       145
62001      2415
103335     4701
144669     5893
186003     3660
227337     1779
268671     1206
310005      582
351339      385
392673      243
434007      139
475341       93
516675       62
558009       41
599343       24
640677       22
682011       23
723345        9
764679        5
806013        5
847347        7
888681        2
930015        2
971349        2
1012683       5
1054017       1
1095351       5
1136685       1
1178019       1
1219353       1
1260687       2
1302021       1
1343355       1
1426023       1
1550025       1
1591359       1
1715361       3
2211369       1
2252703       1
Name: pepole_class, dtype: int64

In [15]:
# Присвоим кредитам более конкретные категории в зависимсоти от приобретаемой услуги
colum=[]
for row in df['lemma_purpose']:
    if 'автомобиль' in row:
        colum.append('автомобиль')
    elif 'жиль' in row:
        colum.append('недвижимость')
    elif 'недвижимость' in row:
        colum.append('недвижимость')
    elif 'свадьба' in row:
        colum.append('свадьба')
    elif 'образование' in row:
        colum.append('образование')
    else:
        colum.append('пропустил')
df['target'] = colum
# Проверю, что пропустил
df.groupby('target')['target'].count()

target
автомобиль       4308
недвижимость    10814
образование      4014
свадьба          2335
Name: target, dtype: int64

**Вывод**

Группа с минимальной З.П. составила 20 667₽  
Группа с макисмальной З.П. составила 2 252 703₽  
Самая часто всречаемая группа людей получает 144 669₽  
Деньги в городе есть.

Чаще всего люди берут кредит на покупку жилья, это хорошо квартиру за границу вывезти сложно.  
Реже всего на свадьбу, услуги тамады нынче не дорогие.

## Шаг 3. Ответьте на вопросы

In [24]:
# Подготовимся
c_d = df[['children','children','debt', 'family_status', 'pepole_class', 'target']].reset_index(drop = True)
c_d.columns = ['children_count','children','debt', 'Семейное положение', 'Уровень дохода', 'Цель кредита']
count_cred = c_d.shape[0] #количество выданых кредитов
c_d.loc[c_d.children != 0, 'children'] = 'Есть дети'
c_d.loc[c_d.children == 0, 'children'] = 'Нет детей'
c_d.loc[c_d.debt == 1, 'debt'] = 'Есть просрочки'
c_d.loc[c_d.debt == 0, 'debt'] = 'Нет просрочек'
#c_d.rename(columns={'children': 'Количество детей', 'debt': 'Просрочка', 'family_status': 'Семейное положение', 'pepole_class': 'Уровень дохода', 'target': 'Цель кредита' }, inplace=True)
c_d.columns = ['Количество детей','Наличие детей','Просрочка','Семейное положение','Уровень дохода','Цель кредита']

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
data_pivot = c_d.pivot_table(index='Наличие детей', columns='Просрочка', values='Уровень дохода', aggfunc='count').fillna(0)
data_pivot['Всего выдали кредитов'] = (data_pivot['Есть просрочки'] + data_pivot['Нет просрочек'])
data_pivot['Доля без просрочек'] = data_pivot['Нет просрочек'] / data_pivot['Всего выдали кредитов']
data_pivot['Доля c просрочками'] = data_pivot['Есть просрочки'] / data_pivot['Всего выдали кредитов']
data_pivot.sort_values(by = 'Доля без просрочек', ascending = False)

Просрочка,Есть просрочки,Нет просрочек,Всего выдали кредитов,Доля без просрочек,Доля c просрочками
Наличие детей,,,,,
Нет детей,1063,13044,14107,0.924647,0.075353
Есть дети,678,6686,7364,0.907930,0.092070


**Вывод**

Можно сказать, что зависимости нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
data_pivot = c_d.pivot_table(index='Семейное положение', columns='Просрочка', values='Уровень дохода', aggfunc='count')
data_pivot['Всего выдали кредитов'] = (data_pivot['Есть просрочки'] + data_pivot['Нет просрочек'])
data_pivot['Доля без просрочек'] = data_pivot['Нет просрочек'] / data_pivot['Всего выдали кредитов']
data_pivot.sort_values(by = 'Доля без просрочек', ascending = False)

Просрочка,Есть просрочки,Нет просрочек,Всего выдали кредитов,Доля без просрочек
Семейное положение,,,,
вдовец / вдова,63,896,959,0.934307
в разводе,85,1110,1195,0.928870
женат / замужем,931,11413,12344,0.924579
гражданский брак,388,3775,4163,0.906798
Не женат / не замужем,274,2536,2810,0.902491


**Вывод**

Можно сказать, что зависимости нет.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
data_pivot = c_d.pivot_table(index='Уровень дохода', columns='Просрочка', values='Семейное положение', aggfunc='count').fillna(0)
data_pivot['Всего выдали кредитов'] = (data_pivot['Есть просрочки'] + data_pivot['Нет просрочек'])
data_pivot['Доля без просрочек'] = data_pivot['Нет просрочек'] / data_pivot['Всего выдали кредитов']
data_pivot.sort_values(by = 'Доля без просрочек', ascending = False)

Просрочка,Есть просрочки,Нет просрочек,Всего выдали кредитов,Доля без просрочек
Уровень дохода,,,,
2252703,0.0,1.0,1.0,1.000000
1054017,0.0,1.0,1.0,1.000000
930015,0.0,2.0,2.0,1.000000
888681,0.0,2.0,2.0,1.000000
1095351,0.0,5.0,5.0,1.000000
1136685,0.0,1.0,1.0,1.000000
764679,0.0,5.0,5.0,1.000000
1178019,0.0,1.0,1.0,1.000000
682011,0.0,23.0,23.0,1.000000


**Вывод**

Можно сказать, что зависимости нет.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
data_pivot = c_d.pivot_table(index='Цель кредита', columns='Просрочка', values='Семейное положение', aggfunc='count')
data_pivot['Всего выдали кредитов'] = (data_pivot['Есть просрочки'] + data_pivot['Нет просрочек'])
data_pivot['Доля без просрочек'] = data_pivot['Нет просрочек'] / data_pivot['Всего выдали кредитов']
data_pivot.sort_values(by = 'Доля без просрочек', ascending = False)

Просрочка,Есть просрочки,Нет просрочек,Всего выдали кредитов,Доля без просрочек
Цель кредита,,,,
недвижимость,782,10032,10814,0.927686
свадьба,186,2149,2335,0.920343
образование,370,3644,4014,0.907823
автомобиль,403,3905,4308,0.906453


**Вывод**

Судя по всему цели на возврат кредита не влияют.

## Шаг 4. Общий вывод

В целом можно сказать, что в данном городе, все люди возвращают кредиты примерно одинаково и их можно раздавать всем подряд.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.